In [1]:
import requests
import threading
import pandas as pd
import flask
from flask import Flask, request
import geopy
from geopy import Nominatim
from translate import Translator
from langdetect import detect
from iso639 import languages
from textblob import TextBlob

In [2]:
key = 'fsq3bR9nCSrR/WbzD82rlvh990Q70wuc8BuuRs0Ypm6fx+w='

# this method helps us get long and lat of certain city

def get_latlong(b):

    
    try:
    
        city = b

        geolocator = Nominatim(user_agent = 'Safra')
    
        loc = geolocator.geocode(city)
         
        # by default
    except: return 'No results found' , f'{b}'
    
    return loc.latitude , loc.longitude

In [3]:
# this is a test
lat , long = get_latlong('jeddah')

In [4]:
lat , long

(21.5810088, 39.1653612)

In [5]:
def translate(array):
        
    # this list contains new translated descriptions
    tlds = []
    
    for desc in array:
        try:
            # Specifying the language for
            # detection
            # dbt : detection before translation
            dbt = detect(desc)

            # saving desc into text to translate
            if desc != 'Not Available':
                text = desc

                blob = TextBlob(text)

                # tat : text after translation
                tat = blob.translate(from_lang = detect(desc) , to = 'en')
                 # if description is already in english ---> save original description
                if dbt != 'en':
                    tlds.append(str(tat))

            # if not --> save translated description
            else:
                tlds.append(desc)
        except: tlds.append(desc)
        
    return tlds

In [6]:
def extract_categories(array):
    
    templist = []
    for element in array():
        
        if element != [] or element 
    templist = [array[i][0]['name'] for i in range(len(array))]
    
    
    return templist
    

In [55]:
def add_photos(array):
    
    
    # list of links
    lol = []
    headers = {
            "accept": "application/json",
            "Authorization": "fsq3bR9nCSrR/WbzD82rlvh990Q70wuc8BuuRs0Ypm6fx+w="
                }
    # index
    index = 0
    for fsq_id in df['fsq_id']:


        url = f"https://api.foursquare.com/v3/places/{fsq_id}/photos?limit=5&sort=POPULAR&classifications=outdoor"

        response = requests.get(url, headers=headers).json()
        print(response)
        if response != []:
            links = ''
            for i in response:
                index = 0
                links += response[index]['prefix']+'original'+response[index]['suffix']+','
                index+=1
            lol.append(links)    
        else:
            lol.append('Not Available')
        


            
        return lol

In [43]:
def retrieve_places(a , c):

    """
    a : condition --- >  example : coffee , art gallery , etc ...
    c : city name
    """
    
    
    lat , long = get_latlong(c)
    if type(lat) != str:

        if a != '':
            fields_url = f"https://api.foursquare.com/v3/places/search?ll={lat}%2C{long}&query={a}&fields=fsq_id%2Cname%2Ctel%2Cprice%2Crating%2Cdescription%2Clocation%2Ccategories"

        else:
            fields_url = f"https://api.foursquare.com/v3/places/search?ll={lat}%2C{long}&fields=fsq_id%2Cname%2Ctel%2Cprice%2Crating%2Cdescription%2Clocation%2Ccategories%2Cfeatures"


        url = fields_url

        headers = {
            "Accept": "application/json",
            "Authorization": key,
            "Accept-Language": 'en'
        }

        response = requests.get(url, headers=headers)

        data = response.json()

        df = pd.json_normalize(data['results'])

        #deleting unnecessary columns

        try:
            df.drop(df.columns.difference(['fsq_id', 'name', 'price', 'rating', 'tel'
                                           ,'location.country', 'location.region'
                                           , 'description' , 'categories']),1,inplace=True)

        except: df = df
            
        
        
        """
        in this phase we add empty columns if columns are not available already
        
        ============== remember to change range when changing number of retrieved rows ==============
        """
        if 'price' not in df.columns:
            df.insert(len(df.columns) , 'price' , ['Not Available' for i in range(df.shape[0])] )
        
        if 'description' not in df.columns:
            df.insert(len(df.columns) , 'description' , ['Not Available' for i in range(df.shape[0])])
        
        if 'rating' not in df.columns:
            df.insert(len(df.columns) , 'rating' , ['Not Available' for i in range(df.shape[0])])

        
#         # renaming columns
        if 'location.country' in df.columns and 'location.region' in df.columns:
            df.rename(columns = {'location.country':'country' , 'location.region':'region'}, inplace = True)
    
        


        # filling nan values

        df = df.fillna('Not Available')


        # translating process starts here
        # error handling
        if 'description' in df.columns:

            # extracting
            array = df['description'].to_list()

            # tdl : translated descriptions list

            """
            in this line we call function to translate all descriptions as following

            other than english ---> translate

            Not Available ---> keep it as it is

            english description ---> keep it as it's
            """ 
            
            tdl = translate(array)

            # insert it into last 
            df.insert(df.columns.get_loc('description')+1  , 'translated_description' , tdl)

#         if 'name' in df.columns:
#             array_n = df['name'].to_list()
            
#             tnl = translate(array_n)
            
#             df.insert(df.columns.get_loc('name')+1 , 'translated_name' , tnl)

        
        if 'region' in df.columns:
            array_r = df['region'].to_list()
            
            trl = translate(array_r)
            
            df.insert(df.columns.get_loc('region')+1 , 'translated_region' , trl)

        if 'categories' in df.columns:
            templist = df['categories'].to_list()
            templist = extract_categories(templist)
            
            df.drop(['categories'] , inplace = True , axis = 1)
            df.insert(len(df) , 'categories' , templist)
            
        try:
            # changing datatypes
            df = df.astype({'price': str , 'rating': str})
        except: df = df
        
        
        '''
        adding photos to dataframe 
        
        ==== i suggest to make this function separately rather th
        lol : ---> list of links
        '''
#         lol = add_photos(df['fsq_id'].to_list())
        
#         df.insert(len(df) , 'photo_url' , lol)
        
        data = df.to_json(orient = 'records')
        return df , data
    else:
        return lat , long 
    

In [44]:
'''

test field


'''

'\n\ntest field\n\n\n'

In [48]:
df, data_json = retrieve_places('' , 'london')
df

C:\Users\Abdullah\AppData\Local\Temp\ipykernel_21036\1004487.py:36: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(df.columns.difference(['fsq_id', 'name', 'price', 'rating', 'tel'


0    [{'id': 16000, 'name': 'Landmarks and Outdoors...
1    [{'id': 16016, 'name': 'Fountain', 'icon': {'p...
2    [{'id': 10004, 'name': 'Art Gallery', 'icon': ...
3    [{'id': 13025, 'name': 'Wine Bar', 'icon': {'p...
4    [{'id': 16000, 'name': 'Landmarks and Outdoors...
5    [{'id': 17076, 'name': 'Liquor Store', 'icon':...
6    [{'id': 16017, 'name': 'Garden', 'icon': {'pre...
7    [{'id': 13032, 'name': 'Cafes, Coffee, and Tea...
8    [{'id': 13383, 'name': 'Steakhouse', 'icon': {...
9    [{'id': 10026, 'name': 'Indie Movie Theater', ...
Name: categories, dtype: object


,fsq_id,description,translated_description,name,translated_name,rating,tel,country,region,translated_region,categories,price
0,4ac518cef964a520f9a520e3,Trafalgar Square is a public space and tourist...,Trafalgar Square is a public space and tourist...,Trafalgar Square,Trafalgar Square,9.2,020 7983 4750,GB,Greater London,Greater London,Landmarks and Outdoors,Not Available
1,50aa7c98e4b0d042f9f0bd59,Not Available,Not Available,East Trafalgar Square Fountain,East Trafalgar Square Fountain,9.1,Not Available,GB,Greater London,Greater London,Fountain,Not Available
2,4ac518cdf964a520e6a520e3,"Located in Trafalgar Square, the National Gall...","Located in Trafalgar Square, the National Gall...",National Gallery,National Gallery,9.4,020 7747 2885,GB,Greater London,Greater London,Art Gallery,1.0
3,4acdb591f964a520c7cc20e3,Not Available,Not Available,Gordon's Wine Bar,Gordon's Wine Bar,9.3,020 7930 1408,GB,Greater London,Greater London,Wine Bar,2.0
4,4c0ce9597189c928961ed6b6,Not Available,Not Available,Whitehall Gardens,Whitehall Gardens,9.2,Not Available,GB,Greater London,Greater London,Landmarks and Outdoors,Not Available
5,572f6ec7498eb9a06b8c640c,Not Available,Not Available,The Whisky Exchange,The Whisky Exchange,9.2,020 7100 0088,GB,Greater London,Greater London,Liquor Store,Not Available
6,4b9e3908f964a520f6d236e3,Not Available,Not Available,Victoria Embankment Gardens,Victoria Embankment Gardens,9.2,020 7641 6000,GB,Greater London,Greater London,Garden,Not Available
7,585970d56ad73d0218d2eb31,"Home of the Cinnamon Social, Ole & Steen St Ja...","Home of the Cinnamon Social, Ole & Steen St Ja...",Ole & Steen,Ole & Steen,9.2,020 3828 8242,GB,Greater London,Greater London,"Cafes, Coffee, and Tea Houses",1.0
8,565ca96e498e87528d10e88c,Not Available,Not Available,Flat Iron,Flat Iron,9.2,Not Available,GB,Greater London,Greater London,Steakhouse,2.0
9,4ad09ceff964a520bdd820e3,The Best God Damned Cinema On The Planet,The Best God Damned Cinema On The Planet,Prince Charles Cinema,Prince Charles Cinema,9.2,020 7494 3654,GB,Greater London,Greater London,Indie Movie Theater,Not Available


In [17]:
df['photos']

0    https://fastly.4sqi.net/img/general/original/1...
1    https://fastly.4sqi.net/img/general/original/5...
2    https://fastly.4sqi.net/img/general/original/2...
3    https://fastly.4sqi.net/img/general/original/7...
4    https://fastly.4sqi.net/img/general/original/K...
5    https://fastly.4sqi.net/img/general/original/3...
6    https://fastly.4sqi.net/img/general/original/1...
7    https://fastly.4sqi.net/img/general/original/4...
8    https://fastly.4sqi.net/img/general/original/1...
9    https://fastly.4sqi.net/img/general/original/5...
Name: photos, dtype: object

In [52]:

    
# list of links
lol = []
headers = {
        "accept": "application/json",
        "Authorization": "fsq3bR9nCSrR/WbzD82rlvh990Q70wuc8BuuRs0Ypm6fx+w="
            }
# index
index = 0
for fsq_id in df['fsq_id']:


    url = f"https://api.foursquare.com/v3/places/{fsq_id}/photos?limit=5&sort=POPULAR&classifications=outdoor"
    
    response = requests.get(url, headers=headers).json()
    print(response)
    if response != []:
        links = ''
        for i in response:
            index = 0
            links += response[index]['prefix']+'original'+response[index]['suffix']+','
            index+=1
        lol.append(links)    
    else:
        lol.append('Not Available')
        



[{'id': '51daca44498ea3dac20a931b', 'created_at': '2013-07-08T14:18:44.000Z', 'prefix': 'https://fastly.4sqi.net/img/general/', 'suffix': '/19397008_mpdBSDOL_tnC9js-qR6MK8SvVfRCZ9n9S_DjW1JGEP0.jpg', 'width': 959, 'height': 717, 'classifications': ['outdoor']}, {'id': '51ad2187498e0a7bb6b0021d', 'created_at': '2013-06-03T23:06:47.000Z', 'prefix': 'https://fastly.4sqi.net/img/general/', 'suffix': '/43219376_8Ug-MlJ9ibHWFFiFgR_sdvRgf9IwKwgN8tE-5KsHan0.jpg', 'width': 612, 'height': 612, 'classifications': ['outdoor']}, {'id': '51f589ea498ee4351f0a878d', 'created_at': '2013-07-28T21:15:22.000Z', 'prefix': 'https://fastly.4sqi.net/img/general/', 'suffix': '/3438798_9dj2oBOX7vo-ztvYrnLEK1jGcIqIsxq1-82Fscm4uJk.jpg', 'width': 612, 'height': 612, 'classifications': ['outdoor']}, {'id': '51890106498efc401697678a', 'created_at': '2013-05-07T13:26:30.000Z', 'prefix': 'https://fastly.4sqi.net/img/general/', 'suffix': '/20132778_OJ1zaHeklTDhSzbv8TH8XoBBlM9L5JEH9AxQevmgQiw.jpg', 'width': 612, 'height'

[{'id': '5d000317112c6c002cb28489', 'created_at': '2019-06-11T19:37:59.000Z', 'prefix': 'https://fastly.4sqi.net/img/general/', 'suffix': '/445965852_rlUPvjt2gJ_KXW3KhGEqOMAxIOrP3nmD2gpnHjokULc.jpg', 'width': 1079, 'height': 1920, 'classifications': ['outdoor']}, {'id': '5b7152a547f876002c09a681', 'created_at': '2018-08-13T09:43:01.000Z', 'prefix': 'https://fastly.4sqi.net/img/general/', 'suffix': '/70643681_sbQ2QvIfnHfYfcTFLlBjKmrgMAjlfY1WGTkHVCFpnwM.jpg', 'width': 1440, 'height': 1920, 'classifications': ['outdoor']}]
[{'id': '5e0e2e1eea67c9000764a579', 'created_at': '2020-01-02T17:53:34.000Z', 'prefix': 'https://fastly.4sqi.net/img/general/', 'suffix': '/14121371_dvP6H995QqElNZYOq5UTXwuniPBgrBifAXRL84VbyyI.jpg', 'width': 1920, 'height': 1440, 'classifications': ['outdoor']}, {'id': '5db2f82c6eac3900086b46e8', 'created_at': '2019-10-25T13:27:08.000Z', 'prefix': 'https://fastly.4sqi.net/img/general/', 'suffix': '/73166987_uYGNNNm1BbeGNTif5Sm-fypWnFKh6kGfFj7a16q7jvI.jpg', 'width': 1440

In [54]:
lol[0]


'https://fastly.4sqi.net/img/general/original/19397008_mpdBSDOL_tnC9js-qR6MK8SvVfRCZ9n9S_DjW1JGEP0.jpg,https://fastly.4sqi.net/img/general/original/19397008_mpdBSDOL_tnC9js-qR6MK8SvVfRCZ9n9S_DjW1JGEP0.jpg,https://fastly.4sqi.net/img/general/original/19397008_mpdBSDOL_tnC9js-qR6MK8SvVfRCZ9n9S_DjW1JGEP0.jpg,https://fastly.4sqi.net/img/general/original/19397008_mpdBSDOL_tnC9js-qR6MK8SvVfRCZ9n9S_DjW1JGEP0.jpg,https://fastly.4sqi.net/img/general/original/19397008_mpdBSDOL_tnC9js-qR6MK8SvVfRCZ9n9S_DjW1JGEP0.jpg,'

In [14]:
app = Flask(__name__)


@app.route('/api' , methods = ['GET'])

def index():
    userInputb = str(request.args['query'])
    userInputa = str(request.args['query'])
    df, data_json = retrieve_places(userInputa , userInputb)

    return data_json


if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Sep/2022 17:22:23] "GET / HTTP/1.1" 404 -
C:\Users\Abdullah\AppData\Local\Temp\ipykernel_34628\4038082940.py:35: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(df.columns.difference(['fsq_id', 'name', 'price', 'rating', 'tel'
127.0.0.1 - - [29/Sep/2022 17:22:54] "GET /api?query=riyadh HTTP/1.1" 200 -
